In [12]:
import pandas as pd
from forex_python.converter import CurrencyRates, RatesNotAvailableError

In [17]:
# Read your data into a pandas DataFrame
# Replace 'your_data.csv' with the path to your CSV file
data = pd.read_csv("raw data/joined_data.csv")
data.shape

(812104, 27)

In [7]:
unique_currencies = data['Currency'].dropna().unique()
currency_rates = pd.DataFrame(unique_currencies, columns=['Currency'])

# Function to get the exchange rate for a given currency
def get_exchange_rate(currency):
    if currency == 'USD':
        return 1
    elif currency == 'GBP':
        return 1.24
    elif currency == 'GHS':
        return 0.084
    else:
        currency_converter = CurrencyRates()
        try:
            return currency_converter.get_rate(currency, 'USD')
        except RatesNotAvailableError:
            print(f"Conversion rate not available for {currency} => USD")
            return None

currency_rates['Exchange Rate'] = currency_rates['Currency'].apply(get_exchange_rate)
print(currency_rates)


   Currency  Exchange Rate
0       USD       1.000000
1       EUR       1.090500
2       GBP       1.240000
3       RUB       0.009524
4       BRL       0.203216
5       AUD       0.677969
6       SAR       0.279589
7       BGN       0.565344
8       JPY       0.007542
9       MXN       0.055396
10      TRY       0.051616
11      NZD       0.628667
12      KRW       0.000769
13      UAH       0.000769
14      CAD       0.750900
15      CHF       1.125165
16      THB       0.029360
17      EGP       0.279589
18      SEK       0.097457
19      CLP       1.250226
20      NOK       0.096974
21      LBP       0.007314
22      COP       1.094400
23      ILS       0.279589
24      SGD       0.753972
25      AED       0.007386
26      VND       1.250226
27      ZAR       0.055465
28      RSD       0.096880
29      PEN       1.097800
30      HKD       0.127389
31      GHS       0.084000
32      PLN       0.238118
33      RON       0.223722
34      CZK       0.047372
35      KZT       0.002898
3

In [19]:
def convert_to_usd(row):
    if pd.notna(row['Currency']):
        if row['Currency'] == 'USD':
            return row['Currency'], row['Purchase Amount']
        else:
            # Lookup the conversion rate from the currency_rates DataFrame
            conversion_rate = currency_rates.loc[currency_rates['Currency'] == row['Currency'], 'Exchange Rate'].iloc[0]
            if conversion_rate is None:
                print(f"Conversion rate not available for {row['Currency']} => USD")
                return row['Currency'], None

        # Return the original currency name and the converted amount
        return row['Currency'], (row['Purchase Amount'] * conversion_rate)
    else:
        return None, None

data[['Original Currency', 'Purchase Amount (USD)']] = data.apply(convert_to_usd, axis=1, result_type='expand')

In [15]:
data['Original Currency'].head(100)

0      USD
1     None
2      USD
3      USD
4      USD
      ... 
95     USD
96    None
97    None
98     USD
99     USD
Name: Currency, Length: 100, dtype: object

In [21]:
data.to_csv('converted_currency.csv', index=False)